# 日本語テキストの感情分析 (BERT)
## 日本語対応の感情分析モデル使用例

In [ ]:
# 必要なライブラリのインストール（初回のみ）
# !pip install transformers torch fugashi ipadic

In [ ]:
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import torch  # torch>=2.6.0

# 1. 日本語用モデルとトークナイザーの読み込み
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# 2. 3クラス分類タスク用のモデル（分類器付き）
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label={0: "negative", 1: "neutral", 2: "positive"},
    label2id={"negative": 0, "neutral": 1, "positive": 2}
)
# 3クラス分類用に変更

In [ ]:
# 3. テスト用日本語テキスト
japanese_texts = [
    "この映画は素晴らしかった！主演俳優の演技が最高でした。",  # ポジティブ
    "最悪の商品です。買って後悔した。",  # ネガティブ
    "普通ですね。特に良いとも悪いとも思わない。",  # ニュートラル
    "レストランの料理は美味しかったですが、サービスがちょっと...",  # 混合
    "昨日、東京タワーに行ってきました。景色がきれいでした。"  # 中立（感情なし）
]

In [ ]:
print(torch.__version__)

In [ ]:
# 4. 感情分析関数
def analyze_japanese_sentiment(text):
    # テキストをトークン化
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )
    
    # モデル推論
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 確率に変換
    probs = torch.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probs).item()
    
    # ラベル定義 (3クラス)
    labels = {
        0: "ネガティブ",
        1: "ニュートラル", 
        2: "ポジティブ"
    }
    
    # 結果表示
    print(f"\nテキスト: '{text}'")
    print(f"予測感情: {labels[predicted_class]}")
    print("確率分布:")
    for i, prob in enumerate(probs[0]):
        print(f"{labels[i]}: {prob*100:.1f}%")
    print("="*50)

In [ ]:
# 5. すべてのテキストを分析
for text in japanese_texts:
    analyze_japanese_sentiment(text)

## 日本語BERTモデルについての注意点
1. この例では東北大学の日本語BERTベースモデルを使用
2. 感情分析用にファインチューニングする必要があります（この例では未調整）
3. より精度の高い日本語感情分析には以下がおすすめ：
   - `daigo/bert-base-japanese-sentiment` (感情分析用ファインチューニング済み)
   - `koheiduck/bert-japanese-finetuned-sentiment`

In [ ]:
# 6. (オプション) ファインチューニング済みモデルを使用する場合
# !pip install sentencepiece
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# 
# finetuned_model = 'daigo/bert-base-japanese-sentiment'
# tokenizer = AutoTokenizer.from_pretrained(finetuned_model)
# model = AutoModelForSequenceClassification.from_pretrained(finetuned_model)
# 
# # 同じanalyze_japanese_sentiment関数が使用可能